# Module INM378/IN3031: Digital Signal Processing and Audio Programming
## Lab 2 - Digital Signals

### 0.  Setting up dependencies 
This step is for installing the latest version of the dsp_ap library code from GitHub.

In [ ]:
try:
    import google.colab
    import subprocess
    p = subprocess.run(['git', 'rev-parse', '--is-inside-work-tree'], stdout=subprocess.PIPE, universal_newlines=True)
    if p.stdout != 'true\n':
        !git clone --depth 1 -q https://github.com/jpauwels/city_dsp_ap.git
        %cd city_dsp_ap
    else:
        !git pull
except:
    %cd city_dsp_ap

After installation we can import some helper classes from the `dsp_ap.signals` package and `numpy`.

In [ ]:
from dsp_ap.signals import AudioSignal, Spectrum
from dsp_ap.operations import apply_adsr, quantise
import numpy as np
from scipy import signal

### 1. Aliasing

Extend the Python code below to generate harmonic and inharmonic sounds (by adding additional sinusoids). Change the frequency parameter `freq1` and the `samplerate` to create aliasing for both cases. What happens to harmonic sounds with aliasing?

In [ ]:
freq1 = 1500 # freq in Hz
amp1 = 0.5 # amplitude factor 
samplerate = 8000 # sample rate in Hz
duration = 0.8 # sound duration in seconds
timepoints = np.arange(samplerate * duration) / samplerate
samples1 = amp1 * np.sin(2 * np.pi * freq1 * timepoints) # samples vector
signal1 = AudioSignal(samples1, samplerate)
signal1.play()
signal1.display(x_range=(0, 0.03))
Spectrum(signal1).display(y_range=(-100, 50))

### 2. Waveform shapes

So far, we've only generated sine waves, which consist of just a single frequency component. Now we're going to have a look at some other, commonly used waveforms in audio, namely:

1. rectangular or square waves
2. triangular waves
3. sawtooth waves

All of these contain multiple frequency components, infinitely many even according to their mathematical definition (of course not in practice). 

We are going to use the [`signal`](https://docs.scipy.org/doc/scipy/reference/signal.html#waveforms) module from the [`scipy`](https://scipy.org/) library to help us generate these waveforms. This module has already been imported into our namespace as `signal` so be careful to not overwrite it by naming one of your variables `signal`. If you still happen to do this, just execute the cell with the `import` statements at the top of this notebook again.

More specifically, take a look at the help for [`signal.square`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.square.html) (for square waves) and [`signal.sawtooth`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html) (both for triangular and sawtooth waves) to learn how to generate these waveforms. Plot them and their spectra and listen to their sound. Below you can find some common parameters and an example for a sine wave to get you started.

#### Common parameters

In [ ]:
freq = 500 # freq in Hz
samplerate = 44100 # sample rate in Hz
duration = 1.5 # sound duration in seconds
timepoints = np.arange(samplerate * duration) / samplerate

#### Sine wave

In [ ]:
sine_samples = np.sin(2 * np.pi * freq * timepoints)
sine_signal = AudioSignal(sine_samples, samplerate)
sine_signal.play()
sine_signal.display(x_range=(0, 0.03))
Spectrum(sine_signal).display(y_range=(10, 40))

#### Rectangular or square wave

#### Triangular wave

#### Sawtooth wave

### 3. Applying an ADSR envelope

Natural sounds evolve over time, unlike the perfectly periodic waveforms we've been generating so far. A natural signal typically has an impulsive start, takes some time to settle and finally fades out. We can approximate this by applying an ADSR (attack-decay-sustain-release) curve (an _envelope_) to the gain (amplitude) of a signal. This curve is defined by its four sections, which are completely defined by the maximum amplitude (which gets reached at the end of the attack), the sustain amplitude and the duration of attack, decay and release. The sustain simply lasts for as long as needed to achieve the requested signal duration.

Below you can find an example of an ADSR curve applied to a sine wave. Listen to it and try applying the same envelope to the other waveform shapes you created above.

#### ADSR parameters

In [ ]:
max_amp = 1 # amplitude
sustain_amp = .4 # amplitude

attack = 0.2 # seconds
decay = 0.3 # seconds
release = 0.5 # seconds

In [ ]:
sine_signal.display()
sine_signal.play()
adsr_signal = apply_adsr(sine_signal, max_amp, sustain_amp, attack, decay, release)
adsr_signal.display()
adsr_signal.play()

### 4. Amplitude modulation

Applying an ADSR envelope is just one way of controlling the gain of a signal. We can even modify the gain of a signal with another periodic signal. We call this technique _amplitude modulation (AM)_, which you might recognise from analogue radios. It was in fact the first technique used to transmit audio by radio waves.

In digital signal processing, we can achieve this effect by multiplying the samples of a base signal (the _carrier_) elementwise by the samples of another signal of the same length (the _modulator_). You can simply use the `*` operator for this.

Below you can find the code for a carrier sine wave of $500 Hz$. Create a modulator sine wave of $3 Hz$ and multiply them together to get an amplitude-modulate waveform. Plot the waveform and listen to it. Try changing the amplitude, frequency and shape of either wave. The frequency of the modulator is typically orders of magnitude smaller than the frequency of the carrier, below the limits of hearing, so don't worry if you can't hear the modulator in itself, only its effect on the carrier.

#### Common parameters

In [ ]:
samplerate = 44100 # % sample rate in Hz
duration = 2 # sound duration in seconds
timepoints = np.arange(duration*samplerate) / samplerate # sample times

#### Carrier wave

In [ ]:
freq1 = 500 # fundamental frequency in Hz
amp1 = 1 # amplitude
carrier_samples = amp1 * np.sin(2 * np.pi * freq1 * timepoints)
carrier_signal = AudioSignal(carrier_samples, samplerate)
carrier_signal.play()
carrier_signal.display()

#### Modulator wave

#### Amplitude modulated wave

### 5. Resampling

Changing the samplerate $f_s$ of a signal (called _resampling_) also changes the Nyquist frequency $f_{\mathrm{Nyq}} = \dfrac{f_s}{2}$ of that signal. When downsampling (meaning that the new samplerate is lower than the old samplerate), a consequence is that the signal might now contain frequency components that can no longer be represented correctly (i.e. all frequencies higher than $f_{\mathrm{Nyq}}$). Aliassing will cause all these frequencies higher than the Nyquist frequency to be reflected into the downsampled spectrum, which is clearly undesirable.

To avoid this, resampling algorithms apply a lowpass filter to remove everything below the new Nyquist frequency before performing the actual downsampling. By removing the unrepresentable frequencies in this way, we ensure that they do not alias into the new spectrum.

A straightforward downsampling algorithm for integer downsampling factors is available as [`signal.decimate`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.decimate.html). Its parameter `n` controls the downsampling filter. Try passing a value of $0$ to switch the filtering off and see what happens, by plotting the spectrum and listening to the result. Use `ftype='fir'` to select a FIR filter. Then resample again with the lowpass filtering on (use `n=None` for a sensible default value) and compare with the previous plot and sound. Repeat the whole process with a signal frequency that falls below the new Nyquist frequency and see what happens.

In [ ]:
samplerate_old = 8000
samplerate_new = 4000
duration = 0.5
timepoints = np.arange(duration*samplerate_old) / samplerate_old # sample times
freq = 2500 # fundamental frequency in Hz, in between the old and new Nyquist frequency
samples = np.sin(2*np.pi*freq*timepoints)

In [ ]:
original_signal = AudioSignal(samples, samplerate_old)
original_signal.play()
Spectrum(original_signal).display(y_range=(-100, 50), title='Original samplerate')

### 6. Amplitude quantisation

Not just the time axis of digital signals is quantised, the amplitude axis is also limited in the number of different values it can represent. This is indicated by the bit-depth, where a bit-depth of 16 means that the signal's amplitude has $2^{16} = 65,536$ possible values.

Sixteen bits give enough precision for the quantisation noise to be inaudible. Quantisation noise is digital noise that arises from the difference between real and quantised waveforms. Lowering the bit-depth will make the digital noise become audible.

Try out quantisation by using the function `quantise(signal, sample_depth)` on some of the signals created above. The result will be a new `AudioSignal`, so you can just call its `play()` method to play it back (as done above). First try quantising to 16 bit and verify there is no audible difference (although our computer representation of the samples uses 64 bits, so the quantisation applied is already quite significant). Then lower the quantisation even further. The sound of the noise depends on the signal, so try playing with waveform shape and frequency.

### 7. A real-world example of data size

The CD standard is defined as having a sampling rate of 44100 Hz, a bit-depth of 16, and 2 channels. How much data does it need 
1. per second?
2. per minute?
3. per hour?

How much space is needed for the sound data of a CD (74 min). What levels of dynamics do 8, 16, 24 bit systems reproduce?

_answer here_

### 8. Square waves in practice

The square wave as a mathematical function has an infinite series of relatively strong overtones. You may have noticed that the rectangular waves sound a bit rough when we play them on our computers. What is the reason? Could a low-pass filter applied to the generated signal help? Why could or could it not?  

_answer here_

### 9. Reading
Read [Chapter 3](http://www.dspguide.com/ch3.htm) of *The Scientist and Engineer's Guide to
Digital Signal Processing* by S. Smith, and Appendices [A.4 Vectors and Matrices (p. 154)](https://archive.org/details/IntroductionToSoundProcessing/page/n161), [A.5 Exponentials and Logarithms (p. 158)](https://archive.org/details/IntroductionToSoundProcessing/page/n165), [A.6 Trigonometric Functions (p. 161)](https://archive.org/details/IntroductionToSoundProcessing/page/n168) of *Introduction to Sound Processing* by D. Rocchesso.